In [1]:
# Data processing
import iris
import numpy as np
import warnings
warnings.filterwarnings('ignore', module='iris')
from iris.experimental import stratify
from pathlib import Path
## Scientific parameters
M_air = 28.97 # molar mass of dry air [g mol-1]
# Coefficients to convert mass mixing ratio to volume mixing ratio
coeff_ch4 = (M_air/16.0425)*1e9
coeff_c2h6 = (M_air/30.0690)*1e12
coeff_c3h8 = (M_air/44.0956)*1e12
coeff_o3 = (M_air/47.997)*1e9
coeff_oh = (M_air/17.007)*1e12
coeff_meoo = (M_air/47.0333)*1e12
coeff_etoo = (M_air/61.0599)*1e12
coeff_proo = (M_air/75.0865)*1e12
coeff_no = (M_air/30.006)*1e9
coeff_no2 = (M_air/46.006)*1e9
coeff_meono2 = (M_air/77.0394)*1e12
coeff_etono2 = (M_air/91.0660)*1e12
coeff_prono2 = (M_air/105.0926)*1e12

#### Read data

In [2]:
# Choose UKCA run
ukca_run_name = 'xojnc' # xojnl, xojnh, xojni, xojnc
path_to_ukca = Path.home() / 'UEA' / 'PhD' / 'models' / 'ukca' / 'raw' / ukca_run_name

In [3]:
# Choose variables
rh = False
ox = False
nox = True
rono2 = False
if rh:
    cb_ch4 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_ch4.nc'))*coeff_ch4
    cb_c2h6 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_c2h6.nc'))*coeff_c2h6
    cb_c3h8 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_c3h8.nc'))*coeff_c3h8
if ox:
    cb_o3 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_o3.nc'))*coeff_o3
    cb_oh = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_oh.nc'))*coeff_oh
#     cb_meoo = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_meoo.nc'))*coeff_meoo
#     cb_etoo = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_etoo.nc'))*coeff_etoo
#     cb_nproo = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_nproo.nc'))*coeff_proo
#     cb_iproo = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_iproo.nc'))*coeff_proo
if nox:
    cb_no = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_no.nc'))*coeff_no
    cb_no2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_no2.nc'))*coeff_no2
if rono2:
    cb_meono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_meono2.nc'))*coeff_meono2
    cb_etono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_etono2.nc'))*coeff_etono2
    cb_nprono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_nprono2.nc'))*coeff_prono2
    cb_iprono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_nprono2.nc'))*coeff_prono2 # nprono2 only when xojnc!

In [4]:
# Load surface_altitude coordinate
orography = iris.load_cube(str(path_to_ukca.parent / 'um_orography_xnvtj.nc'))
alt_cb = iris.util.squeeze(orography)
alt_coord = iris.coords.AuxCoord(alt_cb.data, standard_name='surface_altitude', units='m')
# Add surface_altitude coordinate to a cube
if rh:
    cb_ch4.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_c2h6.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_c3h8.add_aux_coord(alt_coord, data_dims=[2,3])
if ox:
    cb_o3.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_oh.add_aux_coord(alt_coord, data_dims=[2,3])
#     cb_meoo.add_aux_coord(alt_coord, data_dims=[2,3])
#     cb_etoo.add_aux_coord(alt_coord, data_dims=[2,3])
#     cb_nproo.add_aux_coord(alt_coord, data_dims=[2,3])
#     cb_iproo.add_aux_coord(alt_coord, data_dims=[2,3])
if nox:
    cb_no.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_no2.add_aux_coord(alt_coord, data_dims=[2,3])
if rono2:
    cb_meono2.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_etono2.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_nprono2.add_aux_coord(alt_coord, data_dims=[2,3])
    cb_iprono2.add_aux_coord(alt_coord, data_dims=[2,3])
# Derive altitude coordinate from surface_altitude
if rh:
    factory_ch4 = iris.aux_factory.HybridHeightFactory(delta=cb_ch4.coord('level_height'), sigma=cb_ch4.coord('sigma'), 
                                                       orography=cb_ch4.coord('surface_altitude'))
    factory_c2h6 = iris.aux_factory.HybridHeightFactory(delta=cb_c2h6.coord('level_height'), sigma=cb_c2h6.coord('sigma'), 
                                                        orography=cb_c2h6.coord('surface_altitude'))
    factory_c3h8 = iris.aux_factory.HybridHeightFactory(delta=cb_c3h8.coord('level_height'), sigma=cb_c3h8.coord('sigma'), 
                                                        orography=cb_c3h8.coord('surface_altitude'))
if ox:
    factory_o3 = iris.aux_factory.HybridHeightFactory(delta=cb_o3.coord('level_height'), sigma=cb_o3.coord('sigma'), 
                                                      orography=cb_o3.coord('surface_altitude'))
    factory_oh = iris.aux_factory.HybridHeightFactory(delta=cb_oh.coord('level_height'), sigma=cb_oh.coord('sigma'), 
                                                      orography=cb_oh.coord('surface_altitude'))
#     factory_meoo = iris.aux_factory.HybridHeightFactory(delta=cb_meoo.coord('level_height'), sigma=cb_meoo.coord('sigma'), 
#                                                         orography=cb_meoo.coord('surface_altitude'))
#     factory_etoo = iris.aux_factory.HybridHeightFactory(delta=cb_etoo.coord('level_height'), sigma=cb_etoo.coord('sigma'), 
#                                                         orography=cb_etoo.coord('surface_altitude'))
#     factory_nproo = iris.aux_factory.HybridHeightFactory(delta=cb_nproo.coord('level_height'), sigma=cb_nproo.coord('sigma'), 
#                                                         orography=cb_nproo.coord('surface_altitude'))
#     factory_iproo = iris.aux_factory.HybridHeightFactory(delta=cb_iproo.coord('level_height'), sigma=cb_iproo.coord('sigma'), 
#                                                         orography=cb_iproo.coord('surface_altitude'))
if nox:
    factory_no = iris.aux_factory.HybridHeightFactory(delta=cb_no.coord('level_height'), sigma=cb_no.coord('sigma'), 
                                                      orography=cb_no.coord('surface_altitude'))
    factory_no2 = iris.aux_factory.HybridHeightFactory(delta=cb_no2.coord('level_height'), sigma=cb_no2.coord('sigma'), 
                                                      orography=cb_no2.coord('surface_altitude'))
if rono2:
    factory_meono2 = iris.aux_factory.HybridHeightFactory(delta=cb_meono2.coord('level_height'), sigma=cb_meono2.coord('sigma'), 
                                                          orography=cb_meono2.coord('surface_altitude'))
    factory_etono2 = iris.aux_factory.HybridHeightFactory(delta=cb_etono2.coord('level_height'), sigma=cb_etono2.coord('sigma'), 
                                                          orography=cb_etono2.coord('surface_altitude'))
    factory_nprono2 = iris.aux_factory.HybridHeightFactory(delta=cb_nprono2.coord('level_height'), sigma=cb_nprono2.coord('sigma'), 
                                                           orography=cb_nprono2.coord('surface_altitude'))
    factory_iprono2 = iris.aux_factory.HybridHeightFactory(delta=cb_iprono2.coord('level_height'), sigma=cb_iprono2.coord('sigma'), 
                                                           orography=cb_iprono2.coord('surface_altitude'))
if rh:
    cb_ch4.add_aux_factory(factory_ch4)
    cb_c2h6.add_aux_factory(factory_c2h6)
    cb_c3h8.add_aux_factory(factory_c3h8)
if ox:
    cb_o3.add_aux_factory(factory_o3)
    cb_oh.add_aux_factory(factory_oh)
#     cb_meoo.add_aux_factory(factory_meoo)
#     cb_etoo.add_aux_factory(factory_etoo)
#     cb_nproo.add_aux_factory(factory_nproo)
#     cb_iproo.add_aux_factory(factory_iproo)
if nox:
    cb_no.add_aux_factory(factory_no)
    cb_no2.add_aux_factory(factory_no2)
if rono2:
    cb_meono2.add_aux_factory(factory_meono2)
    cb_etono2.add_aux_factory(factory_etono2)
    cb_nprono2.add_aux_factory(factory_nprono2)
    cb_iprono2.add_aux_factory(factory_iprono2)

#### Relevel model data from level height to tagret levels

In [5]:
# Relevel model data from level height to target levels
tgt_levels = np.arange(0, 14000, 100) # [meters]
if rh:
    cbr_ch4 = stratify.relevel(cb_ch4, cb_ch4.coord('altitude'), tgt_levels, axis='level_height')
    cbr_c2h6 = stratify.relevel(cb_c2h6, cb_c2h6.coord('altitude'), tgt_levels, axis='level_height')
    cbr_c3h8 = stratify.relevel(cb_c3h8, cb_c3h8.coord('altitude'), tgt_levels, axis='level_height')
if ox:
    cbr_o3 = stratify.relevel(cb_o3, cb_o3.coord('altitude'), tgt_levels, axis='level_height')
    cbr_oh = stratify.relevel(cb_oh, cb_oh.coord('altitude'), tgt_levels, axis='level_height')
#     cbr_meoo = stratify.relevel(cb_meoo, cb_meoo.coord('altitude'), tgt_levels, axis='level_height')
#     cbr_etoo = stratify.relevel(cb_etoo, cb_etoo.coord('altitude'), tgt_levels, axis='level_height')
#     cbr_nproo = stratify.relevel(cb_nproo, cb_nproo.coord('altitude'), tgt_levels, axis='level_height')
#     cbr_iproo = stratify.relevel(cb_iproo, cb_iproo.coord('altitude'), tgt_levels, axis='level_height')
if nox:
    cbr_no = stratify.relevel(cb_no, cb_no.coord('altitude'), tgt_levels, axis='level_height')
    cbr_no2 = stratify.relevel(cb_no2, cb_no2.coord('altitude'), tgt_levels, axis='level_height')
if rono2:
    cbr_meono2 = stratify.relevel(cb_meono2, cb_meono2.coord('altitude'), tgt_levels, axis='level_height')
    cbr_etono2 = stratify.relevel(cb_etono2, cb_etono2.coord('altitude'), tgt_levels, axis='level_height')
    cbr_nprono2 = stratify.relevel(cb_nprono2, cb_nprono2.coord('altitude'), tgt_levels, axis='level_height')
    cbr_iprono2 = stratify.relevel(cb_iprono2, cb_iprono2.coord('altitude'), tgt_levels, axis='level_height')

In [6]:
# Rename cubes
if rh:
    cbr_ch4.rename('ch4')
    cbr_c2h6.rename('c2h6')
    cbr_c3h8.rename('c3h8')
    cubelist_rh = [cbr_ch4, cbr_c2h6, cbr_c3h8]
if ox:
    cbr_o3.rename('o3')
    cbr_oh.rename('oh')
#     cbr_meoo.rename('meoo')
#     cbr_etoo.rename('etoo')
#     cbr_nproo.rename('nproo')
#     cbr_iproo.rename('iproo')
    cubelist_ox = [cbr_o3, cbr_oh]#, cbr_meoo, cbr_etoo, cbr_nproo, cbr_iproo]
if nox:
    cbr_no.rename('no')
    cbr_no2.rename('no2')
    cubelist_nox = [cbr_no, cbr_no2]
if rono2:
    cbr_meono2.rename('meono2')
    cbr_etono2.rename('etono2')
    cbr_nprono2.rename('nprono2')
    cbr_iprono2.rename('iprono2')
    cubelist_rono2 = [cbr_meono2, cbr_etono2, cbr_nprono2, cbr_iprono2]

#### Save processed data

In [7]:
# Save releveled data to .nc
path_to_prcd = Path.home() / 'UEA' / 'PhD' / 'models' / 'ukca' / 'processed' / ukca_run_name
if rh:
    iris.save(cubelist_rh, str(path_to_prcd)+f'/{ukca_run_name}_relvl_rh.nc')
if ox:
    iris.save(cubelist_ox, str(path_to_prcd)+f'/{ukca_run_name}_relvl_ox.nc')
if nox:
    iris.save(cubelist_nox, str(path_to_prcd)+f'/{ukca_run_name}_relvl_nox.nc')
if rono2:
    iris.save(cubelist_rono2, str(path_to_prcd)+f'/{ukca_run_name}_relvl_rono2.nc')